# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from config import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather = pd.read_csv("../WeatherPy/weather.csv")


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

Create a heat map that displays the humidity for every city from Part 1, as in the following image:

In [3]:
weather = weather[weather['Lat'] != 0]
weather = weather.reset_index(drop=True)
weather

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,rikitea,-23.1203,-134.9692,79.12,74,0,17.52,PF,1649640668
1,karratha,-20.7377,116.8463,89.04,27,0,8.70,AU,1649640668
2,baglan,36.1307,68.7083,76.50,33,100,1.10,AF,1649640669
3,tuktoyaktuk,69.4541,-133.0374,3.20,66,20,13.80,CA,1649640669
4,longyearbyen,78.2186,15.6401,10.24,56,0,23.02,SJ,1649640670
...,...,...,...,...,...,...,...,...,...
554,metro,-5.1131,105.3067,83.95,67,80,5.82,ID,1649640888
555,sokolskoye,57.1412,43.1589,38.61,99,100,2.57,RU,1649640889
556,asadabad,34.7824,48.1185,40.95,40,39,3.00,IR,1649640889
557,maromitsa,59.8683,48.0123,36.61,99,100,7.00,RU,1649640889


In [4]:
gmaps.configure(api_key = g_key)
location = weather[["Lat", "Lng"]].astype(float)
humidity = weather["Humidity"].astype(float)

In [5]:
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(location, weights=humidity, 
                                dissipating=False, max_intensity=100,
                                point_radius = 4)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:

hotel_df = weather.loc[(weather["Max Temp"] <85) & (weather["Max Temp"] >75)
                            & (weather["Humidity"] <80) & (weather["Cloudiness"] == 0), :]
hotel_df = hotel_df.reset_index(drop=True)
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,rikitea,-23.1203,-134.9692,79.12,74,0,17.52,PF,1649640668
1,cabo san lucas,22.8909,-109.9124,77.61,67,0,13.80,MX,1649640549
2,tombouctou,20.0000,-3.0000,76.35,7,0,9.26,ML,1649640701
3,ibra,22.6906,58.5334,76.77,26,0,3.00,OM,1649640706
4,araouane,18.9048,-3.5265,79.20,6,0,10.11,ML,1649640711
5,coffs harbour,-30.3000,153.1333,80.13,65,0,10.36,AU,1649640712
6,nelson bay,-32.7167,152.1500,79.09,68,0,6.38,AU,1649640766
7,pokhara,28.2333,83.9833,75.85,50,0,2.35,NP,1649640770
8,broken hill,-31.9500,141.4333,76.71,38,0,10.36,AU,1649640751
9,bilma,18.6853,12.9164,83.10,7,0,4.72,NE,1649640773


### Hotel Map
* Create a dataframe named `hotel_df` that is the df created above only with the pertinent data.  See the prepared code below (`info_box_template`) to determine what data you will need for your final map.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels (aka "lodging") within 5000 meters.
* Utilize the Google Places API to find the first hotel search result.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
# Create dataframe and empty column
hotel_df = pd.DataFrame(hotel_df)
hotel_df["Hotel Name"] = " "
hotel_lat = hotel_df["Lat"]
hotel_lng = hotel_df["Lng"]
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,rikitea,-23.1203,-134.9692,79.12,74,0,17.52,PF,1649640668,
1,cabo san lucas,22.8909,-109.9124,77.61,67,0,13.80,MX,1649640549,
2,tombouctou,20.0000,-3.0000,76.35,7,0,9.26,ML,1649640701,
3,ibra,22.6906,58.5334,76.77,26,0,3.00,OM,1649640706,
4,araouane,18.9048,-3.5265,79.20,6,0,10.11,ML,1649640711,
5,coffs harbour,-30.3000,153.1333,80.13,65,0,10.36,AU,1649640712,
6,nelson bay,-32.7167,152.1500,79.09,68,0,6.38,AU,1649640766,
7,pokhara,28.2333,83.9833,75.85,50,0,2.35,NP,1649640770,
8,broken hill,-31.9500,141.4333,76.71,38,0,10.36,AU,1649640751,
9,bilma,18.6853,12.9164,83.10,7,0,4.72,NE,1649640773,


In [8]:
hotel_name = []

params = {
    "radius": 5000,
    "keyword": "hotel",
    "key": g_key
}

for index, row in hotel_df.iterrows():

    lat = row["Lat"]
    lng = row["Lng"]

    params["location"] = f"{lat},{lng}"

    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    hotel_result = requests.get(base_url, params=params).json()
    
    try:
#         hotel_name.append(hotel_result["results"][0]["name"])
        hotel_df.loc[index, "Hotel Name"] = hotel_result["results"][0]["name"]
    
    except(KeyError, IndexError):
        print("Missing field/result... skipping.")
    
hotel_df

Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,rikitea,-23.1203,-134.9692,79.12,74,0,17.52,PF,1649640668,People ThankYou
1,cabo san lucas,22.8909,-109.9124,77.61,67,0,13.80,MX,1649640549,Medano Hotel and Suites
2,tombouctou,20.0000,-3.0000,76.35,7,0,9.26,ML,1649640701,
3,ibra,22.6906,58.5334,76.77,26,0,3.00,OM,1649640706,المستقبل للشقق الفندقية
4,araouane,18.9048,-3.5265,79.20,6,0,10.11,ML,1649640711,
5,coffs harbour,-30.3000,153.1333,80.13,65,0,10.36,AU,1649640712,Plantation Hotel
6,nelson bay,-32.7167,152.1500,79.09,68,0,6.38,AU,1649640766,Hotel Nelson
7,pokhara,28.2333,83.9833,75.85,50,0,2.35,NP,1649640770,Hotel Karuna
8,broken hill,-31.9500,141.4333,76.71,38,0,10.36,AU,1649640751,Palace Hotel
9,bilma,18.6853,12.9164,83.10,7,0,4.72,NE,1649640773,


In [18]:
# NOTE: Do not change any of the code in this cell
# Analyze the code below for variables but these should be all columns in hotel_df

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]


In [26]:
# Add marker layer ontop of heat map
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(location, weights=humidity, 
                                dissipating=False, max_intensity=74,
                                point_radius = 4)


marker_layer = gmaps.marker_layer(marker_locations, info_box_content=hotel_info)

fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
fig

Figure(layout=FigureLayout(height='420px'))